<h1><center>Извлечение именованных сущностей </center></h1>

<h3><center>(Named Entity Recognition, NER)</center></h3>

## Spacy

In [1]:
!pip -q install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [3]:
import spacy

In [4]:
nlp_eng = spacy.load("en_core_web_sm")

text = "HSE opens a building in Moscow worth 1 million rubles"

for ent in nlp_eng(text).ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_, sep = '\t\t')

HSE		0		3		ORG
Moscow		24		30		GPE
1 million rubles		37		53		MONEY


In [6]:
doc = nlp_eng("Kate studies computer science in Higher School of Economics")

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(*ents)
print()

for i in range(len(doc)):
    print(*[doc[i].text, doc[i].ent_iob_, doc[i].ent_type_], sep='\t\t')


('Kate', 0, 4, 'PERSON') ('Higher School of Economics', 33, 59, 'ORG')

Kate		B		PERSON
studies		O		
computer		O		
science		O		
in		O		
Higher		B		ORG
School		I		ORG
of		I		ORG
Economics		I		ORG


In [9]:
from spacy import displacy

text = "In September 1972, Jobs enrolled at Reed College in Portland, Oregon."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
#displacy.serve(doc, style="ent") # fails in colab

displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [10]:
displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

Модель для русского языка:

In [ ]:
!python -m spacy download ru_core_news_sm

In [12]:
text_ru = "В сентябре 1972 Стив Джобс поступил в колледж в Портленде, штат Орегон."

nlp_ru = spacy.load("en_core_web_sm")
doc = nlp_ru(text_ru)

displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

## Natasha

In [14]:
!pip -q install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 97.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [15]:
import natasha
from natasha import Doc, NewsEmbedding, NewsNERTagger, MorphVocab

In [16]:
emb = NewsEmbedding()

ner_tagger = NewsNERTagger(emb)

In [17]:
text = 'В феврале 1974 года Стив Джобс устроился техником в молодую компанию Атари в Лос-Гатосе (Калифорния)'
markup = ner_tagger(text)
markup.print()

В феврале 1974 года Стив Джобс устроился техником в молодую компанию 
                    PER───────                                       
Атари в Лос-Гатосе (Калифорния)
PER──   LOC───────  LOC─────── 


Есть отдельные парсеры для разных типов сущностей, они все являются обертками над парсером Yargy.

In [18]:
morph_vocab = MorphVocab() #обертка для Pymorphy2

morph_vocab('стекло')

[MorphForm(normal='стекло', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Neut', 'Number': 'Sing', 'Case': 'Nom'}),
 MorphForm(normal='стекло', pos='NOUN', feats={'Animacy': 'Inan', 'Gender': 'Neut', 'Number': 'Sing', 'Case': 'Acc'}),
 MorphForm(normal='стечь', pos='VERB', feats={'VerbForm': 'Fin', 'Aspect': 'Perf', 'Gender': 'Neut', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Ind'})]

Парсер для имен (согласно документации, его лучше применять к спанам текста):

In [19]:
from natasha import NamesExtractor

names_extractor = NamesExtractor(morph_vocab)

text = 'Генеральным директором Pixar является Эд Кэтмелл, а креативный отдел возглавляет Джон Лассетер'
list(names_extractor(text))

[Match(
     start=12,
     stop=22,
     fact=Name(
         first=None,
         last='директором',
         middle=None
     )
 ),
 Match(
     start=38,
     stop=48,
     fact=Name(
         first='Эд',
         last='Кэтмелл',
         middle=None
     )
 ),
 Match(
     start=50,
     stop=51,
     fact=Name(
         first=None,
         last='а',
         middle=None
     )
 ),
 Match(
     start=52,
     stop=62,
     fact=Name(
         first=None,
         last='креативный',
         middle=None
     )
 ),
 Match(
     start=81,
     stop=94,
     fact=Name(
         first='Джон',
         last='Лассетер',
         middle=None
     )
 )]

In [20]:
text = [
    'генеральный директор Эд Кэтмелл',
    'Джон Лассетер',
    'А. С. Пушкин',
    'Лермонтов'
]

for line in text:
    print(names_extractor.find(line))

Match(start=12, stop=20, fact=Name(first=None, last='директор', middle=None))
Match(start=0, stop=13, fact=Name(first='Джон', last='Лассетер', middle=None))
Match(start=0, stop=12, fact=Name(first='А', last='Пушкин', middle='С'))
Match(start=0, stop=9, fact=Name(first=None, last='Лермонтов', middle=None))


Даты:

In [21]:
from natasha import DatesExtractor

dates_extractor = DatesExtractor(morph_vocab)

text = '24.01.2017, 2015 год, 2014 г, 1 апреля, май 2017 г., 9 мая 2017 года'
print(*list(dates_extractor(text)), sep='\n')

Match(start=0, stop=10, fact=Date(year=2017, month=1, day=24))
Match(start=12, stop=20, fact=Date(year=2015, month=None, day=None))
Match(start=22, stop=28, fact=Date(year=2014, month=None, day=None))
Match(start=30, stop=38, fact=Date(year=None, month=4, day=1))
Match(start=40, stop=51, fact=Date(year=2017, month=5, day=None))
Match(start=53, stop=68, fact=Date(year=2017, month=5, day=9))


Деньги:

In [22]:
from natasha import MoneyExtractor

money_extractor = MoneyExtractor(morph_vocab)

text = "$ 23, 100 рублей, 1,565,321 долларов, 34 тыс. евро, тринадцать рублей тридцать две копейки, 13 руб. 32 коп."
print(*list(money_extractor(text)), sep='\n')

Match(start=2, stop=16, fact=Money(amount=23100, currency='RUB'))
Match(start=18, stop=36, fact=Money(amount=1565321, currency='USD'))
Match(start=38, stop=50, fact=Money(amount=34000, currency='EUR'))
Match(start=92, stop=107, fact=Money(amount=13.32, currency='RUB'))


Адреса:

In [23]:
from natasha import AddrExtractor

addr_extractor = AddrExtractor(morph_vocab)

lines = [
    'Россия, Москва, Тверская улица, дом 5, корпус 3',
    '108845, РФ, Приморский край, г. Находка, ул. Добролюбова, 18',
    'поселок Солнченый, ул. Никитская, дом 2'
]
for line in lines:
    print(addr_extractor.find(line), end='\n\n')


Match(start=0, stop=47, fact=Addr(parts=[AddrPart(value='Россия', type='страна'), AddrPart(value='Москва', type=None), AddrPart(value='Тверская', type='улица'), AddrPart(value='5', type='дом'), AddrPart(value='3', type='корпус')]))

Match(start=0, stop=56, fact=Addr(parts=[AddrPart(value='108845', type='индекс'), AddrPart(value='РФ', type='страна'), AddrPart(value='Приморский', type='край'), AddrPart(value='Находка', type='город'), AddrPart(value='Добролюбова', type='улица')]))

Match(start=0, stop=39, fact=Addr(parts=[AddrPart(value='Солнченый', type='посёлок'), AddrPart(value='Никитская', type='улица'), AddrPart(value='2', type='дом')]))



## BERT

Мы воспользуемся кодом из туториала [HuggingFace](https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt)

В нем разбирается обучение модели BERT на данных соревнования CoNLL 2003.

В данных размечены четыре типа сущностей:

- (PER) PERSON

- (LOC) LOCATION

- (ORG) ORGANIZATION

- (MISC) MISCELLANEOUS

In [25]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [26]:
!pip install -q transformers

In [53]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install -q seqeval

In [44]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


Датасет CoNLL 20023:

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

В нем, помимо именованных сущностей, размечены части речи (POS-теги).

In [28]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [29]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

Маппинг для ner_tags:

In [2]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [3]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [4]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


Обучаем свою модель, для начала посмотрим на токенайзер:

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [35]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Сложность заключается в том, что токенайзер разобьет наши слова на сабворды, и нам нужно будет выровнять сабворды с NER-разметкой (и обратно при инференсе):

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

Посмотрим на примере:

In [30]:
labels = raw_datasets["train"][0]["ner_tags"]
print(raw_datasets["train"][0]['tokens'])
print(inputs.tokens())

word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


Токенизация и выравнивание тегов в одной функции:

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [69]:
len(raw_datasets['train'])

14051


Возьмем сабсемплы датасетов для ускорения вычислений:

In [67]:
for split in ['train', 'validation']:
  raw_datasets[split] = (
      raw_datasets[split]
      .select(
          [i for i in range(len(raw_datasets[split])) if not i % 10]
          )
  )

In [70]:
len(raw_datasets['train'])

1405

In [71]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Кроме того, нам нужно добавить паддинг, для этого мы можем воспользоваться модификацией DataCollator для задачи token classification:

In [35]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [36]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

DataCollator добавил паддинг к изначальным токенизированным текстам (id токенов, если быть точными) и меткам токенов:

In [37]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


Оценка качества:

In [13]:
import evaluate

metric = evaluate.load("seqeval")

In [16]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

Испортим теги руками, чтобы посмотреть на метрики:

In [17]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [18]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [19]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [20]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model.config.num_labels

9

In [22]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
